# This is just a small code to check the Keithley connection.

In [1]:
import pyvisa
import numpy as np
import matplotlib.pyplot as plt
import time
rm = pyvisa.ResourceManager()
rm.list_resources()


event_type=pyvisa.constants.EventType.service_request
event_mech=pyvisa.constants.EventMechanism.queue
# FROM THE INSTRUMENT WEB-INTERFACE THE NAME IS
# TCPIP::169.254.207.124::inst0::INSTR

In [2]:
inst = rm.open_resource('TCPIP::169.254.50.53::inst0::INSTR')
'''# Type of event we want to be notified about
event_type = pyvisa.constants.EventType.service_request
# Mechanism by which we want to be notified
event_mech = pyvisa.constants.EventMechanism.queue
inst.enable_event(event_type, event_mech)
'''

'# Type of event we want to be notified about\nevent_type = pyvisa.constants.EventType.service_request\n# Mechanism by which we want to be notified\nevent_mech = pyvisa.constants.EventMechanism.queue\ninst.enable_event(event_type, event_mech)\n'

In [3]:
'''
with rm.open_resource("TCPIP::169.254.207.124::inst0::INSTR") as inst:
    # Type of event we want to be notified about
    event_type = constants.EventType.service_request
    # Mechanism by which we want to be notified
    event_mech = constants.EventMechanism.queue
    inst.enable_event(event_type, event_mech)
    # Instrument specific code to enable service request
    # (for example on operation complete OPC)
    inst.write("*SRE 1")
    inst.write("INIT")
    # Wait for the event to occur
    response = inst.wait_on_event(event_type, 1000)
    inst.assert_response.event.event_type == event_type
    inst.assert_response.timed_out = False
    inst.disable_event(event_type, event_mech)
'''

'\nwith rm.open_resource("TCPIP::169.254.207.124::inst0::INSTR") as inst:\n    # Type of event we want to be notified about\n    event_type = constants.EventType.service_request\n    # Mechanism by which we want to be notified\n    event_mech = constants.EventMechanism.queue\n    inst.enable_event(event_type, event_mech)\n    # Instrument specific code to enable service request\n    # (for example on operation complete OPC)\n    inst.write("*SRE 1")\n    inst.write("INIT")\n    # Wait for the event to occur\n    response = inst.wait_on_event(event_type, 1000)\n    inst.assert_response.event.event_type == event_type\n    inst.assert_response.timed_out = False\n    inst.disable_event(event_type, event_mech)\n'

In [4]:
# Just to check that everything is ok..
print('Resource name  :', inst.resource_name)
print('Resource info  :',inst.resource_info)
print('Resource class :',inst.resource_class)
print('Resource manuf.:',inst.resource_manufacturer_name)

Resource name  : TCPIP0::169.254.50.53::inst0::INSTR
Resource info  : ResourceInfo(interface_type=<InterfaceType.tcpip: 6>, interface_board_number=0, resource_class='INSTR', resource_name='TCPIP0::169.254.50.53::inst0::INSTR', alias=None)
Resource class : INSTR
Resource manuf.: National Instruments


In [5]:
inst.write('*RST')                   #Reset
inst.write('TRAC:CLE')               # clear the dafault buffer "defbuffer1"
print(inst.query('*IDN?'))

KEITHLEY INSTRUMENTS,MODEL 2450,04451367,1.6.7c



In [ ]:
def keithley_voltage_sweep(inst, V_start, V_stop, points, current_compliance = '3e-02', ms_delay = 0.01): #3e-02 30mA of current compliance by default
    # Make a voltage sweep and measure current both with autorange. Voltage goes from V_start to V_stop with points number of points and 
    # starting after ms_delay ms
    inst.write('*RST')                   #Reset
    inst.write('TRAC:CLE')               # clear the dafault buffer "defbuffer1"
    inst.write('SENS:FUNC "CURR"')       # Set current measurements
    inst.write('SENS:CURR:RANG:AUTO ON') # Set current range
    inst.write('SENS:CURR:RSEN OFF')     # Set 2wires (4wires off)
    inst.write('SOUR:FUNC VOLT')         # Set voltage source
    inst.write('SOUR:VOLT:RANG:AUTO ON') # Set voltage range:auto
    inst.write('SOUR:VOLT:ILIM '+ current_compliance)       # Set current compliace
    inst.write('SOUR:SWE:VOLT:LIN '+ str(V_start) +', '+ str(V_stop) +', '+ str(points) +', '+ str(ms_delay)) #('SOUR:SWE:VOLT:LIN -2.5, 2.5, 51, 0.01')
    inst.write('OUTPut ON')              # turns the output on
    inst.write(':INIT')                  # Start!
    #inst.write('*WAI')                   # wait until all commands are complete
    time.sleep(35)
    x=np.array(inst.query_ascii_values('TRAC:DATA? 1, '+ str(points) +', "defbuffer1", SOUR')) # retrieve the voltage readings
    y=np.array(inst.query_ascii_values('TRAC:DATA? 1, '+ str(points) +', "defbuffer1", READ')) # retrieve the current readings
    inst.write('TRAC:CLE') # clear the dafault buffer "defbuffer1"
    return [x,y]
output =  keithley_voltage_sweep(inst, -2.5, 2.5, 51, '1e-2') #-2.5, 2.5, 51, 0.01
hf_IVcurve=plt.plot(output[0], output[1], '.-')
plt.xlabel('Voltage [V]')
plt.ylabel('Current [A]')
plt.grid(visible='true')

In [6]:
def keithley_single_measure(inst, sense = 'CURR', source = 'VOLT', sourcevalue = 1e-2, compliance = 1e-2): # by default it measures current. To measure voltage, insert 'VOLT'
    inst.write('*RST') # Reset
    inst.write('TRAC:CLE') # clear the dafault buffer "defbuffer1"
    inst.write('SENS:'+ sense +':RANG:AUTO ON') # enable autorange
    inst.write('SOUR:FUNC '+ str(source))         # Set source
    inst.write('SOUR:' + str(source) + ' ' + str(sourcevalue))
    inst.write('SOUR:VOLT:ILIM ' + str(compliance))       # Set current compliace
    inst.write('OUTPut ON')              # turns the output on
    #inst.write(':INIT')                  # Start!
    #time.sleep(0.1)
    measurement = inst.query_ascii_values('MEAS:'+ sense +'? ' )#'READ') # Retrieve one measurement of the specified type
    inst.write('OUTPut OFF')              # turns the output on
    inst.write('TRAC:CLE') # Clear default buffer defbuffer1
    return measurement

valore_output_keithley = keithley_single_measure(inst, sense = 'CURR', source = 'VOLT', sourcevalue = -3)[0]
print(valore_output_keithley)
type(valore_output_keithley)

-4.441302e-05


float

In [ ]:
# Make a measurement 0-1V, measure IV
inst.write('*RST')                   #Reset
inst.write('TRAC:CLE') # clear the dafault buffer "defbuffer1"
inst.write('SENS:FUNC "CURR"')       # Set current measurements
inst.write('SENS:CURR:RANG:AUTO ON') # Set current range
inst.write('SENS:CURR:RSEN OFF')     # Set 2wires (4wires off)
inst.write('SOUR:FUNC VOLT')         # Set voltage source
inst.write('SOUR:VOLT:RANG:AUTO ON') # Set voltage range:auto
inst.write('SOUR:VOLT:ILIM 1')       # Set current compliace
inst.write('SOUR:SWE:VOLT:LIN -2.5, 2.5, 51, 0.01')
inst.write(':INIT')                  # Start!


"""
COMMANDS:
# if commands are not grouped (on the same line and with a ; after each command), then the startnig : is optional
# upper case letters are mandatory, lower case letters are optional, but if you choose to put also the lower case ones, all of them have to be inserted
:OUTPut:STATe ON'      # turn the output on
:OUTPut:STATe OFF'      # turn the output off
DISPlay:SCReen SWIPE_USER # to write on screen 
DISPlay:USER1:TEXT "Test in process"
DISPlay:USER2:TEXT "Do not disturb"
:DISPlay:CLEar # clear the display
:SYSTem:BEEPer <frequency, time> # generate a sound Where frequency is the frequency of the sound in Hz (20 to 20,000) and time is the length of the
sound in seconds, an audible sound can be produced with :SYSTem:BEEPer 500, 1
ROUTe:TERMinals FRONt # To change to the front-panel terminals for current measurements
ROUTe:TERMinals REAR # To change to the rear-panel terminals for current measurements
:SENSe:CURRent:RSENse OFF # change to 2-wire sensing for current 
:SENSe:VOLTage:RSENse OFF # change to 2-wire sensing for voltage
:SENSe:RESistance:RSENse OFF # change to 2-wire sensing for resistance
:SENSe:CURRent:RSENse ON # # change to 4-wire sensing for current
:OUTPut:SMODe NORMal # To set the output-off state to normal
:OUTPut:SMODe ZERO # To set the output-off state to zero
:OUTPut:SMODe HIMPedance # To set the output-off state to high impedance
:OUTPut:SMODe GUARd # To set the output-off state to guard
SENS:FUNC "CURR"    # Set current measurements
SOUR:FUNC VOLT     # Set voltage source
SOUR:VOLT:RANG:AUTO ON # Set voltage range:auto
SOUR:VOLT 5 # Set voltage to 5 V?
SENSe:COUNT 5 # makes five readings
SENSe:FUNCtion "CURR" # measures current
SENSe:CURRent:RANGe:AUTO ON # enable autorange

SENSe:FUNCtion "VOLT" # measures voltage
SENSe:CURRent:RANGe:AUTO ON # enable autorange

SOURce:FUNCtion CURR # set current source
SOURce:CURRent 1e-01 # set current to 100 mA
SOURce:CURRent:VLIM 10 # compliance 10 V?
SENSe:COUNT 5

# ------ makes a linear sweep
Set up a linear sweep that sweeps from 0 to 10 V in 21 steps with a source delay of 200 ms.
SOUR:SWE:VOLT:LIN 0, 10, 21, 200e-3 # the sweep command from 0 to 10 in 21 steps/points (stop-start)/steps +1
:INITiate # start the sweep 
*WAI # wait until all commands are complete
TRAC:DATA? 1, 21, "defbuffer1", SOUR, READ # query the source value and measurement readings
# --------

# ------ makes a single measurement
FETCh? "defbuffer1", DATE, READ # Retrieve the date and measurement value for the most recent data captured in defbuffer1.
                                # Example output:
                                # 03/21/2013,-1.375422E-11
                                
# ------ makes a single measurement
TRACe:MAKE "voltMeasBuffer", 10000
MEAS:VOLT? "voltMeasBuffer", FORM, DATE, READ # Create a buffer named voltMeasBuffer. Make a voltage measurement and store it in the buffer
                                              # voltMeasBuffer and return the formatted reading, the date, and the reading elements from the buffer.
                                              # Example output:
                                              # -00.0024 mV,05/16/2014,-2.384862E-06

# ------- make a single measurement

:TRAC:CLEAR # Clear data from the reading buffer.
:COUN 10 # Set the count to 10.
:MEAS? # Make ten measurements.
:TRAC:DATA? 1,10 # Returns the last measurement of ten readings. 
# --------

# ------
OUTPut ON 
TRACe:TRIGger "defbuffer1"
TRACe:DATA? 1, 5, "defbuffer1", SOUR, READ # return the last measurement of 5 readings from buffer "defbuffer1" with source and readings
OUTPut OFF
# ------

TRACe:CLEar # clear the defaul buffer "defbuffer1"

To make a measurement, you send the MEASure:<function>? command. For example, to make a
current measurement, send the command:
:MEASure:CURRent?
To make a voltage measurement, send the command:
:MEASure:VOLTage?
To make a resistance measurement, send the command:
:MEASure:RESistance?


The example below use a 100 kΩ device under test. The code:
• Makes five readings by sourcing 5e-6 A
• Measures voltage with autorange enabled
• Sets the measure units to ohms
• Uses offset compensation
• Retrieves the source and measure values
Even though the measurement units are in ohms, the measurement range is 2 V.
Using SCPI:
Send the following code:
*RST
SENSe:FUNCtion "VOLT"
SENSe:VOLTage:RANGe:AUTO ON
SENSe:VOLTage:UNIT OHM
SENSe:VOLTage:OCOM ON
SOURce:FUNCtion CURR
SOURce:CURRent 5e-6
SOURce:CURRent:VLIM 10
SENSe:COUNT 5
OUTPut ON
TRACe:TRIGger "defbuffer1"
TRACe:DATA? 1, 5, "defbuffer1", SOUR, READ
OUTPut OFF

The example below measures resistance
*RST
SENSe:FUNCtion "RES"
SENSe:RESistance:RANGe:AUTO ON
SENSe:RESistance:OCOMpensated ON
SENSe:COUNt 5
OUTPut ON
TRACe:TRIGger "defbuffer1"
TRACe:DATA? 1, 5, "defbuffer1", SOUR, READ
OUTPut OFF

SOURce:VOLTage:PROTection PROT20 # overvoltage limit to 20 V
SOURCe:CURRent:VLIMit <n> # voltage compliance in V when sourcing current
SOURCe:VOLTage:ILIMit <n> # current compliance in A when sourcing voltage

"""

'''
# Wait for the event to occur
response = inst.wait_on_event(event_type, 1000)
inst.assert_response.event.event_type == event_type
inst.assert_response.timed_out = False
inst.disable_event(event_type, event_mech)
'''

'''
inst.assert_trigger()
time.sleep(0.1)
inst.wait_on_event(event_type, 1000)
#inst.wait_on_event(event_type)#non sembra funzionare...
#inst.write('*WAI')#non sembra funzionare...
'''
time.sleep(30)#ci mettiamo una brutta pezza per ora.
print('Fatto')

Before executing the cell below, wait for the previous code to be executed.
For now, best way to check the code execution is to connect to the web-interface of the instrument and look at it. Last time it was in 169.254.207.124. We can eventually set a fixed IP for it I think.

In [ ]:
# Leggo 10 dati, per vedere se è andato...
#print(inst.query('TRAC:DATA? 1, 10, "defbuffer1", SOUR, READ'))

In [ ]:
x=np.array(inst.query_ascii_values('TRAC:DATA? 1, 51, "defbuffer1", SOUR'))
y=np.array(inst.query_ascii_values('TRAC:DATA? 1, 51, "defbuffer1", READ'))

In [ ]:
hf_IVcurve=plt.plot(x, y, '.-')
plt.xlabel('Voltage [V]')
plt.ylabel('Current [A]')
plt.grid(visible='true')


In [ ]:
pyvisa.__version__